### Differential Gene Expression Analysis

GSE ID: GSE103264

**MLL1 is essential for retinal development**

By removing MLL1 from mouse retinal progenitors, we discovered that MLL1 plays multiplex roles in retinal development by regulating: 
1) progenitor cell proliferation and cell cycle progression; 
2) retinal cell composition; 
3) maintenance of horizontal neurons; 
4) formation of functional synapses between neuronal layers; and 
5) visual function development. 

Altogether, our results suggest that MLL1 is an indispensable regulator for the development of retinal structure and function.

### Experimental Design

MLL1 KO vs WT (of strain: C57Bl6/J at age: P14)

![Image](./data/DE-Page-5.drawio.png)


In [ ]:
### Install Packages

In [23]:
pip install --upgrade jupyterlab

ERROR: Error in parse(text = x, srcfile = src): <text>:1:5: unexpected symbol
1: pip install
        ^


In [ ]:
if (!require("BiocManager", quietly = TRUE))
     install.packages("BiocManager")

 BiocManager::install("DESeq2")

In [ ]:
## Read In the Counts Matrix

In [43]:
data <- read.csv("counts_new.csv")
head(data)

genes,MLL1_KO_Rep1,MLL1_KO_Rep2,MLL1_KO_Rep3,WT_Rep1,WT_Rep2,WT_Rep3
Gnai3,789,713,549,829,761,738
Pbsn,0,0,0,0,0,0
Cdc45,119,70,83,102,77,81
H19,78,47,47,137,160,114
Scml2,42,45,31,52,39,36
Apoh,3,2,1,6,2,8


In [44]:
duplicate_rows <- duplicated(data$genes)
data <- data[!duplicate_rows,]

In [45]:
# Rearrange the Dataframe to add row.names to the datamatrix
row.names(data) <- data$genes
data <- data[,2:7]
head(data)

,MLL1_KO_Rep1,MLL1_KO_Rep2,MLL1_KO_Rep3,WT_Rep1,WT_Rep2,WT_Rep3
Gnai3,789,713,549,829,761,738
Pbsn,0,0,0,0,0,0
Cdc45,119,70,83,102,77,81
H19,78,47,47,137,160,114
Scml2,42,45,31,52,39,36
Apoh,3,2,1,6,2,8


In [46]:
dim(data)

[1] 55255     6

In [47]:
# load the library DEseq2
library(DESeq2)
library(ggplot2)
library(plotly)
library(dendextend)

ERROR: Error: package or namespace load failed for 'DESeq2' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 there is no package called 'locfit'


In [ ]:
head(data)

In [ ]:
# create a colData dataframe

In [ ]:
colData <- data.frame(condition = c("Mutant","Mutant","Mutant","WT","WT","WT"))
colData

In [ ]:
condition = factor(c("Mutant","Mutant","Mutant","WT","WT","WT"))

In [ ]:
condition

In [ ]:
dim(data)

In [ ]:
# Create a DESeq2 dataset object
dds <- DESeqDataSetFromMatrix(countData = data,
                              colData = colData,
                              design = ~ condition)

In [ ]:
# Anatomy of the DDS Object

In [ ]:
# Run the DESeq2 Pipeline
dds <- DESeq(dds)

In [ ]:
results(dds)

### Principle Component Analysis
---
Principal Component Analysis (PCA) is a statistical technique used to identify global patterns in high-dimensional datasets. It is commonly used to explore the similarity of biological samples in RNA-seq datasets. To achieve this, gene expression values are transformed into Principal Components (PCs), a set of linearly uncorrelated features which represent the most relevant sources of variance in the data, and subsequently visualized using a scatter plot.The first Principal Component is aligned with the largest source of variance, the second Principal Component to the largest remaining source of variance and so on. PCA provides insights into the clustering of samples and exposes batch effects as well as confounding variables in the RNAseq experiment.

In [ ]:
vsd <- varianceStabilizingTransformation(dds, blind = TRUE)
pcaData <- plotPCA(vsd, intgroup = "condition", returnData = TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
PCAplot <- ggplot(pcaData, aes(x = PC1, y = PC2, color = group)) + geom_point(size =3) +
    xlab(paste0("PC1: ", percentVar[1], "% variance")) + ylab(paste0("PC2: ", percentVar[2], "% variance")) +
    ggtitle("PCA with VST data") + theme(plot.title = element_text(hjust = 0.5)) + theme_bw() 
ggplotly(PCAplot)

```vsd <- varianceStabilizingTransformation(dds, blind = TRUE):``` This line performs the VST on the DESeqDataSet object dds. VST is used to stabilize the variance across different expression levels and improve the suitability of the data for PCA. The blind = TRUE argument indicates that the transformation should be performed without knowledge of the experimental groups.

```pcaData <- plotPCA(vsd, intgroup = "condition", returnData = TRUE):``` This line performs the PCA on the variance-stabilized data vsd. The intgroup = "condition" argument indicates that the points in the PCA plot should be colored based on the experimental conditions. The returnData = TRUE argument ensures that the PCA results are returned as a data frame.

```percentVar <- round(100 * attr(pcaData, "percentVar")):``` This line calculates the percentage of variance explained by each principal component. The percentVar variable will contain these values.

```PCAplot <- ggplot(pcaData, aes(x = PC1, y = PC2, color = group)) + geom_point(size = 3) + ...:``` This code sets up a scatter plot using the ggplot2 package. The x-axis represents the first principal component (PC1), the y-axis represents the second principal component (PC2), and the points are colored based on the experimental group. The size = 3 argument sets the size of the points.

```xlab(paste0("PC1: ", percentVar[1], "% variance")) and ylab(paste0("PC2: ", percentVar[2], "% variance")):``` These lines set the x-axis and y-axis labels to indicate the percentage of variance explained by PC1 and PC2, respectively.

```ggtitle("PCA with VST data"):``` This line sets the title of the plot to "PCA with VST data".

```theme(plot.title = element_text(hjust = 0.5)) + theme_bw():``` These lines set the theme of the plot, including centering the title and using a black and white color scheme.

```ggplotly(PCAplot):``` This line converts the ggplot object PCAplot into an interactive plot using the ggplotly function from the plotly package. This allows for additional interactive features like zooming and hovering over points to display additional information.

In [ ]:
pcaData

### Hierarchical Clustering: Dendograms
---
Hierarchical clustering algorithm used to cluster genes or samples based on their expression profiles. It creates a dendrogram, which is a tree-like structure that represents the relationships between genes or samples. The algorithm iteratively merges the most similar genes or samples(here samples) based on their expression profiles until all genes or samples are grouped into clusters. The resulting dendrogram provides a visual representation of the clustering patterns.

In [ ]:
dend <- t(scale(assay(dds))) %>% dist %>% hclust(method = "ward.D2") %>% as.dendrogram
  par(mar = c(5,5,1,5)) # Setting margin for the plot
  dend %>% set("branches_lwd", 2) %>% set("branches_k_color", k = 2) %>% set("highlight_branches_lwd") %>%   plot(horiz=TRUE, xlab = "Samples", ylab = "hclust distance")

```dend <- t(scale(assay(dds))) %>% dist %>% hclust(method = "ward.D2") %>% as.dendrogram:``` This line performs the following steps:

- ```assay(dds):``` Extracts the assay data from the DESeqDataSet object dds, which typically represents the normalized and transformed expression values.
- ```scale():``` Applies scaling to the expression data, standardizing the values to have mean 0 and standard deviation 1 across samples.
- ```t():``` Transposes the scaled expression data, so that samples are in columns and genes are in rows.
- ```dist():``` Computes the distance matrix based on the scaled and transposed expression data.
- ```hclust():``` Performs hierarchical clustering on the distance matrix using the "ward.D2" method, which minimizes the variance when merging clusters.
- ```as.dendrogram():``` Converts the hierarchical clustering result into a dendrogram object.
- ```par(mar = c(5,5,1,5)):``` This line sets the margins of the plot, controlling the space for the plot region and axis labels. The values within mar specify the bottom, left, top, and right margins, respectively.

```dend %>% set("branches_lwd", 2) %>% set("branches_k_color", k = 2) %>% set("highlight_branches_lwd") %>% plot(horiz=TRUE, xlab = "Samples", ylab = "hclust distance"):``` This line performs the following steps:

- ```set("branches_lwd", 2):``` Sets the line width of the dendrogram branches to 2.
- ```set("branches_k_color", k = 2):``` Colors the dendrogram branches based on the number of clusters (k), with 2 clusters in this case.
- ```set("highlight_branches_lwd"):``` Highlights the branches in the dendrogram.
- ```plot(horiz=TRUE, xlab = "Samples", ylab = "hclust distance"):``` Plots the dendrogram with a horizontal orientation, and labels the x-axis as "Samples" and y-axis as "hclust distance".

In [ ]:
# Get the results from the DESeq2 Pipeline
results <- results(dds)

In [ ]:
# Inspecting the Results

- **baseMean:** The average expression level of a gene across all samples. It represents the normalized count values (after accounting for library size differences) and is used as a measure of the overall expression of a gene.

- **log2FoldChange:** This field represents the fold change in gene expression between two conditions or groups. It is expressed in log2 scale, which means a positive value indicates upregulation in the first condition compared to the second condition, while a negative value indicates downregulation.

- **lfcSE:** The standard error associated with the log2 fold change estimation. It provides a measure of uncertainty in the fold change calculation.

- **stat:** The test statistic, which is computed based on the log2 fold change and the standard error. It is often used to assess the significance of differential expression.

- **pvalue:** The p-value represents the statistical significance of the differential expression. It indicates the probability of observing the differential expression by chance alone. A smaller p-value suggests a higher level of significance.

- **padj:** The adjusted p-value, also known as the false discovery rate (FDR). It accounts for multiple hypothesis testing and is often used to control for false positives. Genes with a low adjusted p-value are considered significantly differentially expressed.

- **dispersion:** The estimated dispersion parameter for each gene, which describes the biological variability within the samples. It is a crucial parameter in DESeq2 that helps account for the different levels of variability across genes.

In [ ]:
head(results)

In [ ]:
# Extract significant genes based on adjusted p-value threshold
sig_genes <- subset(results, padj < 0.05)
sig_genes

In [ ]:
# Extract Top 20 Differentially Expressed genes
top_20 <- head(sig_genes[order(sig_genes$padj),],20)
top_20

### MA Plots
---
The term "MA" stands for "M-value versus A-value". Here's a breakdown of the components:

- **M-value:** The M-value represents the log-fold change between two conditions or groups. It is calculated as the logarithm (base 2) of the ratio of expression levels in the two conditions. Mathematically, M = log2(expression_condition_1 / expression_condition_2). A positive M-value indicates upregulation in the first condition compared to the second condition, while a negative M-value indicates downregulation.

- **A-value:** The A-value represents the average expression level of a gene, which is calculated as the average of the log-expression values across the two conditions. Mathematically, A = (log2(expression_condition_1) + log2(expression_condition_2)) / 2. The A-value provides information about the overall expression level of a gene, irrespective of the direction of differential expression.

In an MA plot, each gene is represented as a point, where the x-axis corresponds to the A-value and the y-axis corresponds to the M-value. The plot allows you to visually inspect the distribution of gene expression changes across the range of expression levels.

Interpreting an MA plot involves looking for patterns or trends. Typically, genes that are not differentially expressed will be scattered around the center line (M=0), as their M-values will be close to zero. Genes that are differentially expressed will deviate from the center line, with points either above or below indicating up- or downregulation, respectively.

It's important to note that MA plots provide a global overview of gene expression changes and can help identify patterns and potential issues such as outliers. However, they do not provide statistical significance or control for multiple testing.

In [ ]:
# Plot an MA Plot
plotMA(results, alpha = 0.05,
        main = "MA Plot")

## Volcano Plots
---
In the volcano plot, the negative logarithm (base 10) of the adjusted p-value (or another statistical measure of significance) is typically plotted on the y-axis, representing the statistical significance or evidence for differential expression. The log2 fold change (or another measure of effect size) is plotted on the x-axis, representing the magnitude or direction of differential expression. Each point on the plot represents a gene, and its position indicates the level of statistical significance (y-axis) and the magnitude of expression change (x-axis) for that gene.

In [ ]:
# Define a significance threshold for highlighting DE genes
significance_threshold <- 0.05

# Create a data frame with log2 fold change, -log10(p-value), and DE gene indicator
df <- data.frame(logFC = results$log2FoldChange,
                 negLogPval = -log10(results$padj),
                 DE = results$padj < significance_threshold)

# Customize the plot using ggplot2
ggplot(df, aes(x = logFC, y = negLogPval, color = DE)) +
  geom_point(size = 1, alpha = 0.6) +
  scale_color_manual(values = c("gray", "red"), labels = c("Not DE", "DE")) +
  labs(x = "Log2 Fold Change", y = "-log10(p-value)",
       title = "Volcano Plot") +
  theme_minimal()

In [ ]:
library(EnhancedVolcano)

In [ ]:
  EnhancedVolcano(results,
    lab = rownames(results),
    x = 'log2FoldChange',
    y = 'pvalue',
    ylim=c(0,20))

### Counts Plot

In [ ]:
plotCounts(dds, gene=which.min(results$padj), intgroup="condition")

### Heat Maps of Differentially expressed genes
---
In a heat map, each row represents a gene, and each column represents a sample or condition. The expression levels of the genes are depicted using a color gradient, where different colors or shades represent different expression levels. Here, higher expression levels are represented by brighter or warmer colors ( red), while lower expression levels are represented by darker or cooler colors ( blue).

In [ ]:
# Assuming you have already performed differential expression analysis using DESeq2
# and stored the results in a variable called 'dds'

# Load the pheatmap package
library(pheatmap)

# Get normalized counts from the DESeq2 object
mat <- assay(dds)[top_20@rownames,]

# Perform clustering on the samples and genes
rownames(mat) <- row.names(assay(dds)[top_20@rownames,])
colnames(mat) <- colData(dds)$condition
result <- pheatmap(mat, cluster_rows = TRUE, cluster_cols = TRUE)

In [ ]:
## move the PCA to the top

In [ ]:
##pathway analysis

In [ ]:
top_20@rownames

In [ ]:
plotDispEsts(dds)